# DOWNLOAD THE TEXT CORPUS



In [25]:
!wget http://mattmahoney.net/dc/text8.zip -O text8.gz
!gzip -d text8.gz -f

--2021-08-18 19:53:08--  http://mattmahoney.net/dc/text8.zip
Resolving mattmahoney.net (mattmahoney.net)... 67.195.197.24
Connecting to mattmahoney.net (mattmahoney.net)|67.195.197.24|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31344016 (30M) [application/zip]
Saving to: ‘text8.gz’

text8.gz            100%[===================>]  29.89M   724KB/s    in 43s     

2021-08-18 19:53:52 (710 KB/s) - ‘text8.gz’ saved [31344016/31344016]



# Train word2vec

In [ ]:
!pip install gensim

In [ ]:
from gensim import utils

with open('/content/text8','r') as file:
  documents = file.readlines()

class MyCorpus(object):
    def __iter__(self): 
        for line in documents:
            yield utils.simple_preprocess(line)

In [26]:
import gensim.models
sentences = MyCorpus()
cbow_model = gensim.models.Word2Vec(sentences=sentences,
                                       sg = 0,               #sg=0 for cbow model 
                                       window = 25,
                                       size=100,
                                       min_count =3,
                                       workers=20,
                                       iter = 30)

In [ ]:
len(cbow_model.wv.vocab.keys())

70965

In [ ]:
cbow_model.wv.get_vector('hero')

array([ 4.2342343e-03, -6.3597859e-04,  1.5177694e-03,  4.0271697e-03,
       -1.6366059e-03, -2.1562243e-03, -1.7035489e-03,  4.2467387e-03,
        4.3648328e-03,  4.1203238e-03,  2.4644863e-03,  4.6788380e-04,
        1.1679186e-03,  1.5259923e-03,  2.0522736e-03,  1.5807642e-03,
       -2.7152426e-03,  3.9157792e-04,  3.6191933e-03, -4.8470288e-03,
        2.4328982e-03, -2.1191034e-03, -2.8845586e-03, -7.2563114e-04,
       -2.0366893e-03,  4.1270819e-03, -3.3440476e-03, -1.1131061e-03,
       -4.3544401e-03, -4.0144827e-03, -2.1308621e-04,  4.1074161e-03,
        4.1855900e-03, -4.8363856e-03,  3.0531816e-03,  1.9583469e-03,
        2.9957271e-03, -4.1747424e-03, -4.7330414e-03, -3.6663588e-04,
       -2.6560614e-03,  4.8589646e-03,  6.5342721e-04,  4.5918198e-03,
       -4.0629222e-03, -8.6839212e-04, -2.9804178e-03, -4.1059032e-03,
        2.2883359e-03,  4.5952369e-03, -3.0543266e-03,  3.7165163e-03,
        4.4759545e-03, -4.8660594e-03,  4.2628432e-03,  4.0766725e-04,
      

In [ ]:
cbow_model.wv.most_similar('hero',topn=2)

[('skated', 0.42677775025367737), ('androstenedione', 0.381356805562973)]

# **Pretrained word2vec model**

*   Pretrained word2vec model is released by Google. It is trained on Google News corpus.
*   we dont need to download here but if we are running this code on jupiter or other platform we need to download it.



In [ ]:
import gensim.downloader as api

model = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


# word similarity 

In [100]:
!pip install sklearn

**Importing Data**

*   RSS = Reference similarity score 




In [54]:
import pandas as pd
import csv
data=pd.read_csv('/content/wordsim_similarity_goldstandard.txt',sep='\t')
data=pd.DataFrame(data)
data.columns=['Word1','Word2','RSS']
data.head(10)

,Word1,Word2,RSS
0,tiger,tiger,10.00
1,plane,car,5.77
2,train,car,6.31
3,television,radio,6.77
4,media,radio,7.42
5,bread,butter,6.19
6,cucumber,potato,5.92
7,doctor,nurse,7.00
8,professor,doctor,6.62
9,student,professor,6.81


In [111]:

from sklearn.metrics.pairwise import cosine_similarity
#function to get similarity
def get_word_similarity(word_1,word_2,model):
  word_1_vector = model.get_vector(word_1)
  word_2_vector = model.get_vector(word_2)
  similarity = cosine_similarity(word_1_vector.reshape(1,-1),word_2_vector.reshape(1,-1))[0][0]
  return similarity


for index,row in data.iterrows(): 
  data['CSS']=(get_word_similarity(row['Word1'],row['Word2'],model)) #CSS= COSINE SIMILARITY SCORE
data.head()



,Word1,Word2,RSS,CSS,CSS_2
0,tiger,tiger,10.00,0.118132,0.118132
1,plane,car,5.77,0.118132,0.118132
2,train,car,6.31,0.118132,0.118132
3,television,radio,6.77,0.118132,0.118132
4,media,radio,7.42,0.118132,0.118132


# Spearman correlation

In [98]:
#from scipy import stats
from scipy.stats.mstats import spearmanr

CSS=data['CSS']
RSS=data['RSS']

for index,row in data.iterrows(): 
  p = stats.spearmanr(CSS,RSS)
  print(p)

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrR

# Train fast text 

In [ ]:
!pip install fasttext

     |████████████████████████████████| 68 kB 4.1 MB/s 
  Using cached pybind11-2.7.1-py2.py3-none-any.whl (200 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3094160 sha256=9672b33ea7e6b49978f4b3cf1b3c9232072602198360ccff30dd50fed9f53d2a
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [99]:
import fasttext
fassttext_model = fasttext.train_unsupervised(input = '/content/text8',
                                              ws = 5,
                                              model = 'cbow',
                                              dim = 100,
                                              epoch = 5)

## word similarity 

In [109]:

for index,row in data.iterrows(): 
  data['CSS_2']=get_word_similarity(row['Word1'],row['Word2'],model)
data.head()

,Word1,Word2,RSS,CSS,CSS_2
0,tiger,tiger,10.00,0.118132,0.118132
1,plane,car,5.77,0.118132,0.118132
2,train,car,6.31,0.118132,0.118132
3,television,radio,6.77,0.118132,0.118132
4,media,radio,7.42,0.118132,0.118132


# Spearman correlation

In [112]:
CSS=data['CSS_2']
RSS=data['RSS']

for index,row in data.iterrows(): 
  p = stats.spearmanr(CSS,RSS)
  print(p)

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrR